In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

classifier = pipeline("ner", model="stulcrad/CNEC_2_0_robeczech-base")
str_ = "Jste světa znalý muž a víte stejně dobře jako já , že souvislost mezi současnými krutostmi v Jihovýchodní Asii a tou novou bankovní pobočkou hned vedle obchoďáku Zátoka je přímá a bezprostřední ; byl z toho už vzteklý jak uvázaný pes , protože zájemci o hodiny mu úplně narušili jeho denní režim a on si nemohl po obědě ani zdřímnout ."
classifier(str_)

In [ ]:
str_[162:168]

In [12]:
import json
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

def combine_adjacent_name_tokens(results):
    """
    Spojí navazující tokeny jmen do celých jmen.
    
    Args:
        results: Seznam tokenů z NER modelu.
    
    Returns:
        Seznam slovníků obsahujících spojená jména.
    """
    # Typy tokenů pro jména
    person_types = ['pf', 'ps', 'pp', 'pm', 'pc', 'pd' ]
    
    combined_names = []
    current_name = None
    last_index = -1
    
    # Seřazení výsledků podle indexu
    sorted_results = sorted(results, key=lambda x: x['index'])
    
    for token in sorted_results:
        # Rozdělení entity na typ
        entity_type = token['entity'].split('-')[1] if '-' in token['entity'] else ""
        if entity_type in person_types:
            if current_name is None or token['index'] != last_index + 1:
                # Nové jméno
                if current_name:
                    combined_names.append(current_name)
                current_name = {
                    'start': token['start'],
                    'end': token['end']
                }
            else:
                # Přidání dalšího tokenu
                current_name['end'] = token['end']
            
            last_index = token['index']
    
    # Přidání posledního jména
    if current_name:
        combined_names.append(current_name)
        
    return combined_names

def extract_and_mark_names(text, results):
    """
    Ohraničí nalezená jména značkami @@ a ##.
    
    Args:
        text: Původní text.
        results: Výstupy z NER modelu.
    
    Returns:
        Text s označenými jmény.
    """
    
    combined = combine_adjacent_name_tokens(results)
    combined.sort(key=lambda x: x['start'])
    
    marked_text = ""
    last_end = 0
    
    # Přidání @@## k nalezeným jménům
    for name in combined:
        marked_text += text[last_end:name['start']]
        
        original_name = text[name['start']:name['end']]
        marked_text += "@@" + original_name + "##"
        
        last_end = name['end']
    
    marked_text += text[last_end:]
    
    return marked_text

def process_dataset_with_model(input_file, output_file):
    """
    Zpracuje dataset pomocí modelu a uloží označené kontexty do výstupního souboru.
    
    Args:
        input_file: Cesta k vstupnímu JSON souboru.
        output_file: Cesta k výstupnímu textovému souboru.
    """
    # Načtení modelu
    model = AutoModelForTokenClassification.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
    tokenizer = AutoTokenizer.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
    classifier = pipeline("ner", model=model, tokenizer=tokenizer)
    
    # Načtení datasetu
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Zpracování a uložení
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in data:
            context = item["context"]
            results = classifier(context)
            marked_context = extract_and_mark_names(context, results)
            f.write(marked_context + '\n')

if __name__ == "__main__":
    input_file = "mrc-ner.test.154.json"
    output_file = "hokuspokus154.txt"
    process_dataset_with_model(input_file, output_file)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
str = "OLOMOUC 1920 . ARCIBISKUPSKÁ KNIH - A KAMENOTISKÁRNA V OLOMOUCI . NÁKLADEM CYRILLOMETHODĚJSKÉ BOHOSLOVECKÉ FAKULTY ."
model = AutoModelForTokenClassification.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
tokenizer = AutoTokenizer.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
results = classifier(str)
combined__ = combine_adjacent_name_tokens(results)

print(results)
print(combined__)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-gu', 'score': 0.9969068, 'index': 1, 'word': 'ĠO', 'start': 0, 'end': 1}, {'entity': 'I-gu', 'score': 0.9969133, 'index': 2, 'word': 'LOMOUC', 'start': 1, 'end': 7}, {'entity': 'B-ty', 'score': 0.9982508, 'index': 3, 'word': 'Ġ1920', 'start': 8, 'end': 12}, {'entity': 'B-ic', 'score': 0.99747854, 'index': 5, 'word': 'ĠAR', 'start': 15, 'end': 17}, {'entity': 'I-ic', 'score': 0.9979342, 'index': 6, 'word': 'CI', 'start': 17, 'end': 19}, {'entity': 'I-ic', 'score': 0.9980363, 'index': 7, 'word': 'BI', 'start': 19, 'end': 21}, {'entity': 'I-ic', 'score': 0.9980369, 'index': 8, 'word': 'SK', 'start': 21, 'end': 23}, {'entity': 'I-ic', 'score': 0.9980236, 'index': 9, 'word': 'UP', 'start': 23, 'end': 25}, {'entity': 'I-ic', 'score': 0.9980083, 'index': 10, 'word': 'SKÃģ', 'start': 25, 'end': 28}, {'entity': 'I-ic', 'score': 0.99798703, 'index': 11, 'word': 'ĠKNI', 'start': 29, 'end': 32}, {'entity': 'I-ic', 'score': 0.9979876, 'index': 12, 'word': 'H', 'start': 32, 'end': 33}